In [50]:
import torch
from load_data import *
from utils import *
from vae import *
from sklearn.decomposition import PCA
import numpy as np
import safeopt
import GPy

In [51]:
movie_features = load__movies_info()
user_features = load_users_info()
user_movie_ratings = load_user_movie_rating()

In [52]:
avg_score = user_movie_ratings.groupby("movie id").mean()['rating'].sort_values(ascending=False)
movie_features=pd.merge(movie_features,avg_score,on="movie id")

In [53]:
U_Mat = utility_matrix(user_movie_ratings)

In [54]:
# movie_pca, movie_k = reduce_movie_features(movie_features, k=3)
# user_pca, user_k = reduce_user_features(user_features, k=3)
movies = postprocess_movie(movie_features)
users  = postprocess_user(user_features)

In [56]:
movie_model, user_model = load_models()

In [57]:
with torch.no_grad():
    _, movie_k = movie_model(torch.from_numpy(movies).float())
    _, user_k = user_model(torch.from_numpy(users).float())

In [58]:
movie_k = movie_k.numpy()
user_k = user_k.numpy()

In [59]:
norm = np.linalg.norm(movie_k)
movie_k = movie_k/norm
norm = np.linalg.norm(user_k)
user_k = user_k/norm

In [65]:
std = 2

noise_var = 0.1 ** 2
res = []


for user_idx, row in U_Mat.iterrows():

    ratings = np.nan_to_num(row.to_numpy())
    rated_idx = np.nonzero(ratings)
    
    parameters_set = []
    rating_look_up = {}
    for movie_idx in rated_idx[0]:

        v = movie_k[movie_idx].reshape(1, -1)
        u = user_k[user_idx - 1].reshape(1, -1)
        x = (v.T @ u).flatten()
#         print(x)
        rating_look_up[x.sum()] = ratings[movie_idx]
        parameters_set.append(x)
    parameters_set = np.stack(parameters_set)

    init_safe_movie, init_reward = np.argmax(ratings), np.max(ratings)  
    v = movie_k[init_safe_movie].reshape(1, -1)
    u = user_k[user_idx - 1].reshape(1, -1)
    x0 = (v.T @ u).flatten()
    y0 = rating_look_up[x0.sum()]
    
    kernel = GPy.kern.Linear(225, variances=std**2)
#     kernel = GPy.kern.RBF(input_dim=225, variance=std**2, lengthscale=.4, ARD=True)
    likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
    gp = GPy.core.GP(np.array([x0]), np.array([[y0]]), kernel, likelihood)
    opt = safeopt.SafeOpt(gp, parameters_set, fmin=2.5, lipschitz=None, threshold=0.2, beta=1)
    
    explored = [x0.sum()]
    avg_reward = 0
    violation = 0
    
    count = 0
    for i in range(300):
        try:
            x_next = opt.optimize()
        except EnvironmentError:
            continue
        count += 1
        reward = rating_look_up[x_next.sum()]
#         print(x_next.sum(), reward)
        avg_reward += reward
        if x_next.sum() not in explored:
            print(x_next.sum(), reward)
            explored.append(x_next.sum())
            
            if reward < 2.5:
                violation += 1
        
        opt.add_new_data_point(x_next, reward)
    res.append([violation / count, avg_reward / count, len(explored) / len(np.where(ratings > 2.5)[0])])
    print('user:', str(user_idx), 'violation:', str(violation), 'avg reward', str(avg_reward / count), 'percent explored:', str(len(explored) / len(np.where(ratings > 2.5)[0])))
    if user_idx == 50:
        break

-0.0001456726 5.0
-4.543233e-05 4.0
-0.000112347414 1.0
8.989737e-06 5.0
1.2541401e-05 2.0
-8.6325745e-05 5.0
-9.3888346e-05 4.0
-7.392702e-05 3.0
-8.846569e-05 5.0
user: 1 violation: 2 avg reward 4.833333333333333 percent explored: 0.045662100456621
6.72022e-07 3.0
1.0197618e-06 3.0
1.2321689e-06 4.0
1.4276884e-06 5.0
2.0457228e-06 5.0
-6.7273504e-07 1.0
2.3082175e-06 4.0
1.0597969e-06 4.0
1.4630932e-06 5.0
1.2570381e-06 5.0
2.3759785e-06 5.0
3.847672e-07 4.0
2.3655448e-06 4.0
2.2380846e-06 3.0
2.3912653e-06 4.0
user: 2 violation: 1 avg reward 4.83 percent explored: 0.2807017543859649
-8.197018e-05 4.0
-0.00013151205 3.0
-6.321035e-05 3.0
-0.00013434941 5.0
-1.4533653e-05 1.0
-7.5689284e-05 4.0
8.037547e-06 1.0
-0.00013872303 2.0
-5.4515112e-05 2.0
-3.6563742e-05 1.0
-7.101742e-05 2.0
-0.00011573947 4.0
-7.857622e-05 1.0
-0.00014888808 3.0
user: 3 violation: 7 avg reward 3.4166666666666665 percent explored: 0.5
-5.858849e-05 3.0
-6.988541e-05 5.0
-7.445537e-05 5.0
-1.8790772e-05 4.0
-

In [70]:
p_vio = np.stack(res)[:, 0]
avg_reward = np.stack(res)[:, 1]
p_explored = np.stack(res)[:, 2]

In [74]:
p_vio.mean(), p_vio.std()

(0.027702564102564105, 0.07908668269282564)

In [75]:
avg_reward.mean(), avg_reward.std()

(4.232858119658119, 0.7426172184651606)

In [76]:
p_explored.mean(), p_explored.std()

(0.2156727062199699, 0.1468844624172138)

In [42]:
std = 2

noise_var = 0.05 ** 2

for user_idx, row in U_Mat.iterrows():

    ratings = np.nan_to_num(row.to_numpy())
    rated_idx = np.nonzero(ratings)
    
    parameters_set = []
    rating_look_up = {}
    for movie_idx in rated_idx[0]:

        v = movie_k[movie_idx]
        u = user_k[user_idx - 1]
        x = np.dot(v, u)
        rating_look_up[x] = ratings[movie_idx]
        parameters_set.append(x)
    parameters_set = np.stack(parameters_set)
    
    parameters_set = parameters_set.reshape(-1, 1)

    init_safe_movie, init_reward = np.argmax(ratings), np.max(ratings)  
    v = movie_k[init_safe_movie]
    u = user_k[user_idx - 1]
    x0 = np.dot(v, u)
    y0 = rating_look_up[x0]
    
    
#     gp = GPy.models.GPRegression(x0, y0, kernel, noise_var=noise_var)
#     kernel = GPy.kern.Linear(400, variances=std**2)
    kernel = GPy.kern.RBF(input_dim=1, variance=std**2, lengthscale=.4, ARD=True)
    likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
    gp = GPy.core.GP(np.array([[x0]]), np.array([[y0]]), kernel, likelihood)
    opt = safeopt.SafeOpt(gp, parameters_set, fmin=2.5, lipschitz=None, threshold=2.5, beta=1.5)
    
    explored = [x0]
    avg_reward = 0
    violation = 0

    for i in range(300):
        x_next = opt.optimize()

        reward = rating_look_up[x_next[0]]
        
        if x_next.sum() not in explored:
#             print(x_next, reward)
            explored.append(x_next)
            avg_reward += reward
            if reward < 2.5:
                violation += 1
        
        opt.add_new_data_point(x_next, reward)

    print('user:', str(user_idx), 'violation:', str(violation), 'avg reward', str(avg_reward / len(explored) - 1), 'percent explored:', str(len(explored) / len(np.where(ratings > 2.5)[0])))
 

user: 1 violation: 1 avg reward 2.9285714285714284 percent explored: 0.0639269406392694
user: 2 violation: 3 avg reward 2.0 percent explored: 0.19298245614035087
user: 3 violation: 2 avg reward 1.625 percent explored: 0.26666666666666666
user: 4 violation: 1 avg reward 1.0 percent explored: 0.13043478260869565
user: 5 violation: 4 avg reward 2.4285714285714284 percent explored: 0.25225225225225223
user: 6 violation: 1 avg reward 2.789473684210526 percent explored: 0.10734463276836158
user: 7 violation: 1 avg reward 3.2142857142857144 percent explored: 0.07547169811320754
user: 8 violation: 2 avg reward 2.6923076923076925 percent explored: 0.26
user: 9 violation: 0 avg reward 2.3333333333333335 percent explored: 0.14285714285714285
user: 10 violation: 0 avg reward 3.3888888888888893 percent explored: 0.09782608695652174
user: 11 violation: 4 avg reward 2.56 percent explored: 0.15822784810126583
user: 12 violation: 0 avg reward 2.6666666666666665 percent explored: 0.12
user: 13 violation

OSError: There are no safe points to evaluate.

In [22]:

U_Mat.columns

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
           dtype='int64', name='movie id', length=1682)

In [13]:
U_Mat

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for user_idx, row in U_Mat.iterrows():
    
    ratings = np.nan_to_num(row.to_numpy())
    rated_idx = np.nonzero(ratings)
    
    # construct parameter sets, movie_k and user_k are training features. 
    # They are obtained other with a VAE or NMF. We use k=15
    parameters_set = []
    rating_look_up = {}
    for movie_idx in rated_idx[0]:
        v = movie_k[movie_idx].reshape(1, -1)
        u = user_k[user_idx - 1].reshape(1, -1)
        x = (v.T @ u).flatten()
        rating_look_up[x.sum()] = ratings[movie_idx]
        parameters_set.append(x) 
    parameters_set = np.stack(parameters_set)

    # init safe seed by picking a highest rated movie by the user
    init_safe_movie, init_reward = np.argmax(ratings), np.max(ratings)  
    v = movie_k[init_safe_movie].reshape(1, -1)
    u = user_k[user_idx - 1].reshape(1, -1)
    x0 = (v.T @ u).flatten()
    y0 = rating_look_up[x0.sum()]
    
    kernel = GPy.kern.Linear(225, variances=std**2)
    likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
    gp = GPy.core.GP(np.array([x0]), np.array([[y0]]), kernel, likelihood)
    opt = safeopt.SafeOpt(gp, parameters_set, fmin=2.5, lipschitz=None, threshold=0.2)
    
    
    for i in range(300):
        x_next = opt.optimize()
        reward = rating_look_up[x_next.sum()]       
        opt.add_new_data_point(x_next, reward)